In [ ]:
!pip install peft
!pip install datasets
!pip install accelerate -U
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 11.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-many

In [ ]:
import os
import random
import string
import torch
import time
import numpy as np
import transformers
from peft import LoraConfig, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, GenerationConfig, TrainingArguments, Trainer
from datasets import Dataset, load_dataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [ ]:
#loading model and the tokenizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name, torch_dtype=torch.bfloat16)
pretrained_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [ ]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    (print(f'Number of trainable parameters: {trainable_model_params}'))
    return trainable_model_params

In [ ]:
pretrained_model_params = print_number_of_trainable_model_parameters(pretrained_model)

In [ ]:
prompt = """Act as a mental health councelor and answer the following:

I'm going through some things with my feelings and myself.
I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here..
I've always wanted to fix my issues, but I never get around to it.

How can I change my feeling of being worthless to everyone?

Response:
"""

inputs = tokenizer(prompt, return_tensors='pt').to(device)

output = tokenizer.decode(
        pretrained_model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )

dash_line = '-'.join('' for x in range(100))

print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'MODEL GENERATION:\n{output}\n')

In [ ]:
huggingface_dataset_name = "Amod/mental_health_counseling_conversations"
dataset = load_dataset(huggingface_dataset_name)

dataset

In [ ]:
def tokenize_function(example):
  # start_prompt = 'Act as a mental health councelor and answer the following:\n\n'
  # end_prompt = '\n\nResponse: '
  # prompt = [start_prompt + context + end_prompt for context in example["Context"]]
  example['input_ids'] = tokenizer(example["Context"], padding="max_length", truncation = True,  return_tensors="pt").input_ids
  example['labels'] = tokenizer(example["Response"], padding="max_length", truncation = True,  return_tensors="pt").input_ids

  return example

In [ ]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
tokenized_datasets

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(['Context','Response'])
tokenized_datasets

In [ ]:
tokenized_datasets = tokenized_datasets.filter(lambda example, index: index % 100 == 0, with_indices=True)
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

In [ ]:
def find_target_modules_for_lora(model):

  layers = set()
  for name, module in model.named_modules():
    if "Linear" in str(type(module)):
      layer_type = name.split('.')[-1]
      layers.add(layer_type)

  return list(layers)

target_modules = find_target_modules_for_lora(pretrained_model)
target_modules

LoRA config

In [ ]:
lora_config = LoraConfig(
    r=16, # Rank
    lora_alpha=16,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type="Causal_LM" # FLAN-T5
)

peft_model = get_peft_model(pretrained_model,
                            lora_config)

peft_model_params = print_number_of_trainable_model_parameters(peft_model)

In [ ]:
print(f'% decrease in params: {((peft_model_params - pretrained_model_params) / pretrained_model_params)*100}')

% decrease in params: -98.57057329069042


In [ ]:
output_dir = f'./peft-dialogue-summary-training-{str(int(time.time()))}'

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-3, # Higher learning rate than full fine-tuning.
    num_train_epochs=1,
    logging_steps=1,
    max_steps=1
)

In [ ]:
trainer = Trainer(
    model=peft_model,
    args= peft_training_args,
    train_dataset=tokenized_datasets['train'],
)

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer.train()

In [ ]:
prompt = """Act as a mental health councelor and answer the following:

I'm going through some things with my feelings and myself.
I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here..
I've always wanted to fix my issues, but I never get around to it.

How can I change my feeling of being worthless to everyone?

Response:
"""

inputs = tokenizer(prompt, return_tensors='pt').to(device)

output = tokenizer.decode(
        pretrained_model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )

dash_line = '-'.join('' for x in range(100))

print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'MODEL GENERATION:\n{output}\n')